In [3]:
import pandas as pd
df =pd.read_csv("ebay_iphone_uncleaned.csv")
df

,Name,Seller,Condition,Price,Shipping Cost,Shipping Country,Free Return,Last Item,Best Seller,Buy Now,Generation
0,Apple iPhone 13 - 128GB - spacegrey (bez simlo...,prywatny,Używany,"576,33","62,32",Niemcy,False,False,False,False,13
1,Apple iPhone 12 - 64GB,prywatny,Używany,"950,00",0,Polska,False,False,False,False,12
2,iPhone 12 Czerwony 64GB Produkt Czerwony Odblo...,firma,Używany,"4,95","74,01",Wielka Brytania,False,False,False,False,12
3,Pakiet 6 iPhone 12 / X / 7 / 6s / 6S Plus z bl...,firma,Tylko na części,"19,35","64,30",Niemcy,False,False,False,False,12
4,iphone 11 128gb,prywatny,Tylko na części,"200,00",0,Polska,False,False,False,False,11
...,...,...,...,...,...,...,...,...,...,...,...
951,Apple iPhone 4 -GB nieznany- Czarny (Trzy) A13...,prywatny,Używany,"49,43","75,96",Wielka Brytania,False,False,False,False,4
952,Apple iPhone 12 128GB czarny niebieski zielony...,firma,Używany,"898,83","107,80",Stany Zjednoczone,False,False,True,False,12
953,Apple iPhone 8 części,prywatny,Tylko na części,"199,90","164,87",Wielka Brytania,False,False,False,False,8
954,Apple iPhone 5 - 16GB - czarny i łupkowy (odbl...,prywatny,Używany,"149,93","73,02",Wielka Brytania,False,False,False,False,5


In [2]:
df.loc[df.Seller=='prywatny']

,Name,Seller,Condition,Price,Shipping Cost,Shipping Country,Free Return,Last Item,Best Seller,Buy Now,Generation


### Poprawa danych

In [39]:
df.info()
df["Seller"]= df["Seller"].astype('category')
df["Condition"]= df["Condition"].astype('category')
df["Shipping Country"]= df["Shipping Country"].astype('category')
df["Shipping Cost"]=df["Shipping Cost"].apply(lambda x : "0" if x in "Bezpłatna wysyłka za granicę" else x)
df["Generation"]= df["Generation"].astype('category')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 956 entries, 0 to 955
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Name              956 non-null    object
 1   Seller            956 non-null    object
 2   Condition         956 non-null    object
 3   Price             956 non-null    object
 4   Shipping Cost     956 non-null    object
 5   Shipping Country  956 non-null    object
 6   Free Return       956 non-null    bool  
 7   Last Item         956 non-null    bool  
 8   Best Seller       956 non-null    bool  
 9   Buy Now           956 non-null    bool  
 10  Generation        940 non-null    object
dtypes: bool(4), object(7)
memory usage: 56.1+ KB


In [40]:
df["Shipping Cost"]=df["Shipping Cost"].str.replace(" ","")
df["Shipping Cost"]=df["Shipping Cost"].apply(lambda x : "0" if x =="Kosztwysyłkiniezostałpodany" else x)
df["Shipping Cost"]=df["Shipping Cost"].str.replace(",",".").replace(" ","").replace("\t","").astype(float)

df["Price"]=df["Price"].str.replace(",",".")

In [41]:
import regex as re
df["Price"]=df["Price"].apply(lambda x : x.split("do")[0] if re.search('[a-zA-Z]',x) else x)
df["Price"]=df["Price"].str.replace(" ","")
df["Price"]=df["Price"].str.replace(",",".")
df["Price"]=df["Price"].astype(float)

In [42]:
df["Generation"].value_counts()

12          180
11          151
se           77
13           74
xr           56
xs           41
7            41
15           41
6s           38
5            35
6            33
8            31
14           23
4s           16
4            16
x            14
5s           13
1.           10
5c            7
3gs           5
3g            4
11,           4
6s,           2
specimen      2
-             2
ze            2
6,            1
5,            1
xs,           1
12,           1
xr,           1
128gb         1
x/xs          1
13,           1
13-           1
se2022        1
1st           1
partia        1
ipod          1
at&t          1
8-            1
8,            1
2g            1
7-            1
7,            1
4/5/6         1
6s-128gb      1
6+,           1
Name: Generation, dtype: int64

### Poprawa nazw generacji

In [43]:
import numpy as np
df["Generation"]=df["Generation"].apply(lambda x: str(x).split(",")[0])
df["Generation"]=df["Generation"].apply(lambda x: str(x).replace(".","").replace("-",""))
df.loc[df["Generation"] == "",["Generation"]] = None

In [44]:
df["Generation"].value_counts()

12          181
11          155
se           77
13           76
xr           57
7            43
xs           42
15           41
6s           40
5            36
6            34
8            33
14           23
4            16
4s           16
nan          16
x            14
5s           13
1            10
5c            7
3gs           5
3g            4
ze            2
specimen      2
se2022        1
6s128gb       1
partia        1
x/xs          1
6+            1
at&t          1
128gb         1
ipod          1
2g            1
4/5/6         1
1st           1
Name: Generation, dtype: int64

### Zajęcie się brakującymi wartościami

In [45]:
import numpy as np
df.isna().sum()
df.dropna(inplace=True)

## Usuwnie odstających wierszy

### Usuwanie obserwacji z generacją występującą tylko 1 raz

In [46]:
x =  df["Generation"].value_counts()
x =x[ x.values ==1].index.to_list()
index_to_drop=df[df["Generation"].isin(x)].index
df.drop(index_to_drop,inplace=True)
df["Generation"].value_counts()

12          181
11          155
se           77
13           76
xr           57
7            43
xs           42
15           41
6s           40
5            36
6            34
8            33
14           23
nan          16
4            16
4s           16
x            14
5s           13
1            10
5c            7
3gs           5
3g            4
ze            2
specimen      2
Name: Generation, dtype: int64

### Usuwanie obserwacji z krajem występującą tylko 1 raz

In [47]:
x =  df["Shipping Country"].value_counts()
x =x[ x.values ==1].index.to_list()
index_to_drop=df[df["Shipping Country"].isin(x)].index
df.drop(index_to_drop,inplace=True)

### Zastąpenie wartości Ceny i Kosztu wysyłki odstających średnią

In [48]:
z = ((abs(df["Price"]-df["Price"].mean())/df["Price"].std()) > 3)
index_to_drop = z[z == True].index
index_to_change=index_to_drop.to_list()
df.loc[index_to_change,["Price"]] = df["Price"].mean()

In [49]:
z = ((abs(df["Shipping Cost"]-df["Shipping Cost"].mean())/df["Shipping Cost"].std()) > 3)
index_to_drop = z[z == True].index
index_to_change=index_to_drop.to_list()
df.loc[index_to_change,["Shipping Cost"]] =df["Shipping Cost"].mean()

### Zapisanie do pliku

In [50]:
df.to_csv("ebay_iphone.csv",index=False)

In [ ]:
df